In [ ]:
from google.colab import drive
import os


In [ ]:

# Mount Google Drive
drive.mount('/content/drive')

# Path to the main folder (klasifikasi_endek_bali) in Drive
main_folder = '/content/drive/MyDrive/klasifikasi_endek_bali/'

# Path to the train and test folders
train_folder = main_folder + 'train/'
test_folder = main_folder + 'test/'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_model_optimization as tfmot

# Load and preprocess your dataset
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Create the base ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Prune the model
end_step = np.ceil(len(train_generator) / 32).astype(np.int32) * 10
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.90, begin_step=0, end_step=end_step)
}
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Train the pruned model
pruned_model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[EarlyStopping(patience=3)])

# Convert the pruned model to a standard Keras model
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Evaluate the final model
test_loss, test_acc = final_model.evaluate(val_generator)
print("Test accuracy:", test_acc)
